In [ ]:
from fastrag.utilities.qdrant_database import QdrantDatabase

In [ ]:
qdrant_database = QdrantDatabase()

In [ ]:
collection_name = "qna_collection"
# qdrant_database.delete_collection(collection_name)
qdrant_database.create_collection(collection_name)

In [ ]:
qdrant_database.load_recent_responses(collection_name=collection_name, limit=10)

In [ ]:
# qdrant_database.add_response(collection_name, "what are you doing?", "I am studying")
qdrant_database.add_response(collection_name, "tell me about Nepal", "It is a beautiful country.")

In [ ]:
import os
from qdrant_client import QdrantClient, models

qdrant_url = os.getenv("QDRANT_URL", "http://0.0.0.0:6333")

qdrant_client = QdrantClient(url=qdrant_url)



In [ ]:
import time

payload = {
    "user": "who are you?",
    "assistant": "I am somebody",
    "timestamp": int(time.time())
}

payload.__str__()


In [ ]:
import uuid

def create_uuid(text: str = None) -> str:
    import hashlib
    hash_bytes = hashlib.sha256(text.encode('utf-8')).digest()
    return str(uuid.UUID(bytes=hash_bytes[:16]))

create_uuid(payload.__str__())

In [ ]:
point_data = models.PointStruct(id=payload['timestamp'], vector=[0], payload=payload)
point_data

In [ ]:
qdrant_client.upsert(
    collection_name=collection_name,
    points=[point_data]
)

In [ ]:
search_results[0]

In [ ]:
search_results = qdrant_client.scroll(collection_name=collection_name, with_payload=True)
search_results
# sorted_responses = sorted(
#     [result.payload for result in search_results[0]],
#     key = lambda x: x["timestamp"],
#     reverse=True
# )

# # return_value = [response["text"] for response in sorted_responses[:10]]
# return_value

In [ ]:
search_results[0]